## Dataset trainticket m9dgg

In [5]:
%reload_ext autoreload
%autoreload 2

In [4]:
DATASET_ID = "m9dgg"

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

In [2]:
import sys
sys.path.append('../../')

from tsdr import tsdr
from eval import groundtruth
from meltria import loader
from eval import validation

In [6]:
metrics_files = !find "/datasets/argowf-chaos-{DATASET_ID}/" -type f -name "*.json"
records = loader.load_dataset(metrics_files, target_metric_types={
        "containers": True,
        "services": True,
        "nodes": True,
        "middlewares": True,
    },
    num_datapoints=4*45, # 45min
)

In [7]:
len(records)

93

In [8]:
well_injected_records = validation.find_records_detected_anomalies_of_sli(records, faulty_datapoints=4*5, sli_index=0)

In [9]:
len(well_injected_records)

3

In [10]:
well_injected_records = validation.find_records_detected_anomalies_of_cause_metrics(well_injected_records, faulty_datapoints=4*5)

In [11]:
len(well_injected_records)

3

In [15]:
import sys
sys.path.append("../")
from notebooklib.save import run_and_save_tsdr_to_each_set, load_tsdr_by_chaos

In [13]:
run_and_save_tsdr_to_each_set(
    DATASET_ID,
    well_injected_records, tsdr_options={
        "step2_clustering_method_name": "hierarchy",
        "step2_hierarchy_dist_threshold": 0.02,  # should be <1.0 if 'sbd' is specified
        "step2_hierarchy_dist_type": "sbd", # 'sbd' or 'hamming'
        "step2_hierarchy_linkage_method": "single",  # 'single','complete','average','weighted', 'centroid', 'median', 'ward'
        "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
        "step2_clustering_choice_method": "medoid",  # 'medoid' or 'maxsum'
    },
    suffix="hierarchy-single_sbd_medoid",
)

In [ ]:
run_and_save_tsdr_to_each_set(
    DATASET_ID,
    well_injected_records, tsdr_options={
        "step2_dbscan_min_pts": 1,
        "step2_dbscan_dist_type": "sbd",  # 'pearsonr' or 'sbd'
        "step2_dbscan_algorithm": "dbscan",  # 'dbscan' or 'hdbscan'
        "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
        "step2_clustering_choice_method": "medoid",  # 'medoid' or 'maxsum'
    },
    suffix="fluxrank_sbd_medoid",
)